# Crawling

In [6]:
headers = {
'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
}

In [7]:
from requests import request, HTTPError
from requests.compat import urljoin,urlparse,quote,unquote
from time import sleep
import re

def download(url, params={},headers={},method='GET',maximum=1):
    resp = request(url=url, params=params if method=='GET' else{},
                   data=params if method =="POST" else{},
                   headers=headers, method= method)
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500< e.response.status_code <600 and maximum > 0:
            printint('다시 시도')
            sleep(1)
            resp= download(url,params,headers,method,maximumum-1)
        print(e)
        
    return resp

In [8]:
def can_fetch(url,path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())

    return  True if path in allow else False, allow, disallow

In [9]:
def new_download(url):
    _ = urlparse(url)
    print(can_fetch(_[0]+'://'+_[1], _[2])[0])
    
    return download(url,headers = headers)

In [11]:
Queue = list()

seed = 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&oq=%ED%8C%8C%EC%9D%B4%EC%8D%AC&aqs=chrome.0.69i59j35i39j0i433j0i20i263i433j0i433j69i61l3.987j0j7&sourceid=chrome&ie=UTF-8'
resp = new_download(seed)

False


In [12]:
from bs4 import BeautifulSoup

dom = BeautifulSoup(resp.text,'lxml')

In [30]:
len(dom.select('a'))

112

In [34]:
_.has_attr('href'), len(dom.select('a[href]'))
#len(dom.select('a:not[href=#]'))
#a 는 112개, 하지만 href는 105개기때문에 둘다 포함한것만 가져와야한다!

(False, 105, 0)

In [37]:
links = [urljoin(seed, _.attrs['href']) for _ in dom.select('a[href]')
        if _.attrs['href'] != '#']

In [38]:
links

['https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&gbv=1&sei=G23mYKD2J_Ks0PEPxfir0Ao',
 'https://support.google.com/websearch/answer/181196?hl=ko',
 'https://www.google.com/webhp?hl=ko&sa=X&ved=0ahUKEwjg7-yywNLxAhVyFjQIHUX8CqoQPAgI',
 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjg7-yywNLxAhVyFjQIHUX8CqoQ_AUoAXoECAEQAw',
 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&source=lnms&tbm=vid&sa=X&ved=2ahUKEwjg7-yywNLxAhVyFjQIHUX8CqoQ_AUoAnoECAEQBA',
 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&source=lnms&tbm=bks&sa=X&ved=2ahUKEwjg7-yywNLxAhVyFjQIHUX8CqoQ_AUoA3oECAEQBQ',
 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&rlz=1C5CHFA_enKR914KR914&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjg7-yywNLxAhVyFjQIHUX8CqoQ_AUoBHoECAEQBg',
 'https://www.google.com/preferences',
 'https://docs.python.org/

In [13]:
def crawling(seed):
    resp = new_download(seed)
    
    #추가작업 -> Content-type: text/*, application/json+xml, image
    dom = BeautifulSoup(resp.text, 'lxml')
    
    return [urljoin(seed, _.attrs['href'])
             for _ in dom.select('a[href]')
             if not _.attrs['href'].startswith('#')] 
    

In [49]:
seed_list=[]
seed_list.append('https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&source=lnms&tbm=nws&sa=X&ved=0ahUKEwjT2JGv2NLxAhX5IjQIHfAMBU4Q_AUIDCgF')

visted_list=[]
count=1

while seed_list:
    seed=seed_list.pop(0) #Queue,FIFO => BFS
    #seed = seed_list.pop(-1) #Stack, LIFO => DFS
    url_list=crawling(seed)
    
    print('{} {}'.format(count, seed))
    
    for _ in url_list:
        if _ not in visted_list and _ not in seed_list:
            seed_list.append(_)
            
    print('찾은 갯수:{} 방문할 곳:{} 방문한 곳:{} '.format(len(url_list), len(seed_list), len(visted_list)))
            
    visted_list.append(seed)
    count+=1
    
    if count>100:
        break

False
1 https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&source=lnms&tbm=nws&sa=X&ved=0ahUKEwjT2JGv2NLxAhX5IjQIHfAMBU4Q_AUIDCgF
찾은 갯수:41 방문할 곳:41 방문한 곳:0 
False
2 https://www.google.com/?sa=X&ved=0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQOwgC
찾은 갯수:15 방문할 곳:53 방문한 곳:1 
False
3 https://www.google.com/?sa=X&gbv=1&output=search&tbm=nws&ved=0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQPAgE
찾은 갯수:9 방문할 곳:60 방문한 곳:2 
False
4 https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&source=lnms&sa=X&ved=0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQ_AUIBygA
찾은 갯수:72 방문할 곳:128 방문한 곳:3 
False
5 https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&tbm=isch&source=lnms&sa=X&ved=0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQ_AUICCgB
찾은 갯수:60 방문할 곳:187 방문한 곳:4 
False
6 https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&tbm=vid&source=lnms&sa=X&ved=0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQ_AUICSgC
찾은 갯수:72 방문할 곳:255 방문한 곳:5 
False
7 https://maps.google.com/maps?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&um=1

False
43 https://mail.google.com/mail/&ogbl
찾은 갯수:4 방문할 곳:1741 방문한 곳:42 
False
44 https://www.google.co.kr/imghp?hl=ko&ogbl
찾은 갯수:9 방문할 곳:1745 방문한 곳:43 
False
45 https://www.google.co.kr/intl/ko/about/products
찾은 갯수:277 방문할 곳:1983 방문한 곳:44 
False
46 https://accounts.google.com/ServiceLogin?hl=ko&passive=true&continue=https://www.google.com/%3Fsa%3DX%26ved%3D0ahUKEwihovvP2dLxAhUSMH0KHfYZDEQQOwgC&ec=GAZAmgQ
찾은 갯수:4 방문할 곳:1982 방문한 곳:45 
False
47 https://about.google/?utm_source=google-KR&utm_medium=referral&utm_campaign=hp-footer&fg=1
찾은 갯수:44 방문할 곳:1997 방문한 곳:46 
False
48 https://www.google.com/intl/ko_kr/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1
찾은 갯수:55 방문할 곳:2035 방문한 곳:47 
False
49 https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1
찾은 갯수:1 방문할 곳:2035 방문한 곳:48 
False
50 https://google.com/search/howsearchworks/?

False
88 https://www.google.com/search?gbv=1&sa=X&q=%EC%9E%90%EB%B0%94&stick=H4sIAAAAAAAAAONgFuLQz9U3MK0yjFfiArHMi5Oy05K0hLOTrfST83Nz8_OsijNTUssTK4sXsbK9mTfh9YYpAKJCBYY2AAAA&ved=2ahUKEwi5nJbR2dLxAhUVOn0KHXezDcwQ0I4BegQICxAa
찾은 갯수:81 방문할 곳:3617 방문한 곳:87 
False
89 https://www.google.com/search?gbv=1&sa=X&q=C%2B%2B&stick=H4sIAAAAAAAAAONgFuLQz9U3MK0yjFcCs7LSC9O1hLOTrfST83Nz8_OsijNTUssTK4sXsTI7a2sDAIk31Y0xAAAA&ved=2ahUKEwi5nJbR2dLxAhUVOn0KHXezDcwQ0I4BegQICxAc
찾은 갯수:92 방문할 곳:3705 방문한 곳:88 
False
90 https://www.google.com/search?gbv=1&sa=X&q=%EC%9E%90%EB%B0%94%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8&stick=H4sIAAAAAAAAAONgFuLQz9U3MK0yjFcCs4wKLM21hLOTrfST83Nz8_OsijNTUssTK4sXsQq9mTfh9YYpb7qWvG1c83rZ3rddOwC1-I4aQAAAAA&ved=2ahUKEwi5nJbR2dLxAhUVOn0KHXezDcwQ0I4BegQICxAe
찾은 갯수:68 방문할 곳:3769 방문한 곳:89 
False
91 https://www.google.com/search?gbv=1&sa=X&q=%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D+%EC%96%B8%EC%96%B4&stick=H4sIAAAAAAAAAONgFuLQz9U3MK0yjFeCsIqMDLSEs5Ot9JPzc3Pz86yKM1NSyxMrixexir2d0vJ64ZxX23e8

-------

In [ ]:
#튜플로 묶는 방식도!
seed_list=[]
seed_list.append(('https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gbv=1&source=lnms&tbm=nws&sa=X&ved=0ahUKEwjT2JGv2NLxAhX5IjQIHfAMBU4Q_AUIDCgF',1))

visted_list=[]
count=1

while seed_list:
    #seed=seed_list.pop(0) #Queue,FIFO => BFS
    seed = seed_list.pop(-1) #Stack, LIFO => DFS
    url_list=crawling(seed[0])
    
    print('{} {}'.format(count, seed))
    
    for _ in url_list:
        if _ not in [link[0] for link in visted_list] \
        and _ not in [link[0] for link in seed_list]:
            seed_list.append((_,seed[1]+1))
            
    print('찾은 갯수:{} 방문할 곳:{} 방문한 곳:{} '.format(len(url_list), len(seed_list), len(visted_list)))
            
    visted_list.append(seed)
    count+=1
    
    if seed[1] > 3:
        break

-----------

------

In [ ]:
더 복잡해진다...!!!
실제로 이미지 저장을 해볼거다

In [14]:
directory = {'text':'TXT',"application":"DOC", "image":"IMG"}
def crawling(seed):
    resp=new_download(seed)
    ########추가작업 -> Content-type : text\* application/json+xml,image
    ctype=resp.headers['Content-type'].split('/')
    if ctype[0] in ['text', 'application', 'image']:
        if ctype[0] == 'text':
            dom=BeautifulSoup(resp.text,'lxml')

            returnlist= list()
            for _ in dom.select('a[href], img[src], iframe[src]'):
                if _ .has_attr('href'):
                    if not _.attrs['href'].startswith('#'):
                        returnlist.append(urljoin(seed,_.attrs['href']))
                else:
                    returnlist.append(urljoin(seed,_.attrs['src']))
            return  returnlist

        path=directory[ctype[0]]
        path+='/'+seed.split('/')[-1]+'.'+ctype[-1]
        with open(path,'wb')as f:
            f.write(resp.content)
            
    return None

In [78]:
seed_list=[]
seed_list.append(('https://comic.naver.com/webtoon/weekdayList?week=thu',1))

visted_list=[]
count=1

while seed_list:
    seed=seed_list.pop(0) #Queue #FIFO => BFS
    #seed=seed_list.pop(-1) #Stack, LIfO =>DFS
    url_list=crawling(seed[0])
    
    print('{} {}'.format(count, seed))
    if url_list:
        for _ in url_list:
            if _ not in [link[0] for link in visted_list] and _ not in [link[0] for link in seed_list]:
                seed_list.append((_,seed[1]+1))
            
        print('찾은 갯수:{} 방문할 곳:{} 방문한 곳:{} '.format(len(url_list), len(seed_list), len(visted_list)))
            
    visted_list.append(seed)
    count+=1
    
    if seed[1]>3: #BFS
        break
        
    #if count>100: #DFS
        #break

False
1 ('https://comic.naver.com/webtoon/weekdayList?week=thu', 1)
찾은 갯수:368 방문할 곳:214 방문한 곳:0 
False
2 ('https://www.naver.com', 2)
찾은 갯수:288 방문할 곳:435 방문한 곳:1 
False
3 ('https://comic.naver.com/index', 2)
찾은 갯수:161 방문할 곳:487 방문한 곳:2 
False
4 ('https://novel.naver.com', 2)
찾은 갯수:199 방문할 곳:659 방문한 곳:3 
False
5 ('https://comic.naver.com/webtoon/list.nhn?titleId=775690', 2)
찾은 갯수:113 방문할 곳:666 방문한 곳:4 
False
6 ('https://comic.naver.com/webtoon/weekday', 2)
찾은 갯수:1414 방문할 곳:1419 방문한 곳:5 
False
7 ('https://comic.naver.com/genre/bestChallenge', 2)
찾은 갯수:212 방문할 곳:1509 방문한 곳:6 
False
8 ('https://comic.naver.com/genre/challenge', 2)
찾은 갯수:212 방문할 곳:1604 방문한 곳:7 
False
9 ('https://comic.naver.com/mypage/favorite', 2)
False
10 ('https://series.naver.com/comic/recommendList.nhn', 2)
찾은 갯수:97 방문할 곳:1685 방문한 곳:9 
False
11 ('https://series.naver.com/novel/recommendList.nhn', 2)
찾은 갯수:125 방문할 곳:1752 방문한 곳:10 
False
12 ('https://comic.naver.com/webtoon/genre', 2)
찾은 갯수:834 방문할 곳:2089 방문한 곳:11 
False

False
79 ('https://comic.naver.com/webtoon/list?titleId=743270&weekday=thu', 2)
찾은 갯수:150 방문할 곳:6007 방문한 곳:78 
False
80 ('https://comic.naver.com/webtoon/list?titleId=733076&weekday=thu', 2)
찾은 갯수:153 방문할 곳:6037 방문한 곳:79 
False
81 ('https://comic.naver.com/webtoon/list?titleId=770873&weekday=thu', 2)
찾은 갯수:145 방문할 곳:6059 방문한 곳:80 
False
82 ('https://comic.naver.com/webtoon/list?titleId=771717&weekday=thu', 2)
찾은 갯수:143 방문할 곳:6080 방문한 곳:81 
False
83 ('https://comic.naver.com/webtoon/list?titleId=742351&weekday=thu', 2)
찾은 갯수:149 방문할 곳:6106 방문한 곳:82 
False
84 ('https://comic.naver.com/webtoon/list?titleId=724482&weekday=thu', 2)
찾은 갯수:153 방문할 곳:6136 방문한 곳:83 
False
85 ('https://comic.naver.com/webtoon/list?titleId=738843&weekday=thu', 2)
찾은 갯수:151 방문할 곳:6164 방문한 곳:84 
False
86 ('https://comic.naver.com/webtoon/list?titleId=758661&weekday=thu', 2)
찾은 갯수:146 방문할 곳:6187 방문한 곳:85 
False
87 ('https://comic.naver.com/webtoon/list?titleId=768471&weekday=thu', 2)
찾은 갯수:145 방문할 곳:6209 방문한 곳:86 
F

404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
146 ('https://shared-comic.pstatic.net/thumb/webtoon/731130/thumbnail/thumbnail_IMAG10_0f5fa6cc-3d7c-4e20-a42d-60b32de309e5.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
147 ('https://shared-comic.pstatic.net/thumb/webtoon/756137/thumbnail/thumbnail_IMAG10_64143a9b-dfdb-4f2e-878d-5f3453a82475.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
148 ('https://shared-comic.pstatic.net/thumb/webtoon/762279/thumbnail/thumbnail_IMAG10_7eca3c20-7c36-49b7-a7af-af933e9c8c09.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
149 ('https://shared-comic.pstatic.net/thumb/webtoon/739503/thumbnail/thumbnail_IMAG10_f515191d-adb2-466c-b014-66b276d8b471.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
150 ('https://shared-comic.pstati

404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
185 ('https://shared-comic.pstatic.net/thumb/webtoon/769662/thumbnail/thumbnail_IMAG10_6cfb32f1-5e0a-45d5-989c-6dce97268b9c.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
186 ('https://shared-comic.pstatic.net/thumb/webtoon/733076/thumbnail/thumbnail_IMAG10_9c070ef4-e569-4a14-b1cf-7ce368c07016.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
187 ('https://shared-comic.pstatic.net/thumb/webtoon/770873/thumbnail/thumbnail_IMAG10_91a40117-9233-42ca-9532-3c730716d479.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
188 ('https://shared-comic.pstatic.net/thumb/webtoon/771717/thumbnail/thumbnail_IMAG10_42d5fd4e-0558-415a-8164-d585df440e66.jpg', 2)
404 Client Error: Not Found for url: https://shared-comic.pstatic.net/robots.txt
False
189 ('https://shared-comic.pstati

239 ('https://dict.naver.com/', 3)
찾은 갯수:185 방문할 곳:10227 방문한 곳:238 


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

--------

In [ ]:
전쟁끝나고 세시..

-------

In [15]:
directory = {'text':'TXT',"application":"DOC", "image":"IMG"}
def crawling(seed):
    resp=new_download(seed)
    ########추가작업 -> Content-type : text\* application/json+xml,image
    ctype=resp.headers['Content-type'].split('/')
    
    returnlist = list()
    
    if ctype[0] in ['text', 'application', 'image']:
        if ctype[0] == 'text':
            dom=BeautifulSoup(resp.text,'lxml')

            for _ in dom.select('a[href], img[src], iframe[src]'):
                if _ .has_attr('href'):
                    if not _.attrs['href'].startswith('#'):
                        returnlist.append(urljoin(seed,_.attrs['href']))
                else:
                    returnlist.append(urljoin(seed,_.attrs['src']))
                    
        path = directory[ctype[0]]
        path += '/'+seed.split('/')[-1]+'.'+ctype[-1]
        path = re.sub(r'?\\','',path)
        
        with open(path,'wb')as f:
            f.write(resp.content)
            
    return None

In [ ]:
seed_list=[]
seed_list.append(('https://comic.naver.com/webtoon/weekdayList?week=thu',1))

visted_list=[]
count=1

while seed_list:
    seed=seed_list.pop(0) #Queue #FIFO => BFS
    #seed=seed_list.pop(-1) #Stack, LIfO =>DFS
    url_list=crawling(seed[0])
    
    print('{} {}'.format(count, seed))
    if url_list:
        for _ in url_list:
            if _ not in [link[0] for link in visted_list] and _ not in [link[0] for link in seed_list]:
                seed_list.append((_,seed[1]+1))
            
        print('찾은 갯수:{} 방문할 곳:{} 방문한 곳:{} '.format(len(url_list), len(seed_list), len(visted_list)))
            
    visted_list.append(seed)
    count+=1
    
    if seed[1]>3: #BFS
        break
        
    #if count>100: #DFS
        #break

----------------------------------------

### PageRank

크롤링과 스크랩핑은 다른 개념이다.
얻고자 하는 목적이 다르다!

크롤링은 기본단위가 웹이다.굉장히 큰 스케일이다
대부분은 스크랩핑을 할거다. 지정한 웹사이트만 긁는것

스크랩핑은 다양한 형태의 데이터를 긁어온다
크롤러는 에이전트만 있으면 된다
스크랩핑은 에이전트와 파서 둘다 있어야 한다! 파서는 편하게 이용하기 위해서 쓰는거지 반드시 필요한 것은 아니다

특정 도메인에서만 가져올수 있게끔. 블로그에서 외부로 링크가 걸려있으면 뛰쳐나가기 때문에
특정 도메인만 허용할수 있다.

동일한 키워드를 가지고 여러 사이트에서 데이터를 가져온다
그럼 어느 사이트에서 가져온 데이터가 더 좋은 데이터일까? - 구글의 페이지 랭크!

제일 중요한게 보통 포털사이트, 뉴스사이트
개별 포스팅,블로그들이 비교적 덜 중요하게 여겨진다.

링크는 단방향성이다 -
내 사이트에서 나가는 링크를 아웃바운드 링크
내 사이트로 들어오는 링크는 인바운드 링크

수식으로 표현 할 수 있다!


focused crawling + scraping 예제
시드 = 뉴스 첫페이지
while 리스트가 없을때까지: =>크롤링
    특정 id, class 에서만 링크를 수집 => 스크래핑

In [95]:
seed_list = list ()
seed_list.append('https://news.naver.com/')

seen_list = list ()

while seed_list:
    seed = seed_list.pop(0)
    
    resp = download(seed, headers=headers)
    
    #Scrapping 영역
    #뉴스만 가져올 것이다
    if not re.search(r'text/html',resp.headers['Content-type'.lower()]):         #뉴스가 아니면 날려주기
        continue
        
    dom = BeautifulSoup(resp.text, 'lxml')
    
    main_area = dom.select_one('#main_content')                    #링크 수집    
    
    if not main_area:
        continue
        
    #뉴스기사에만 해당 되는 애들 가져오기. "구조" 활용
    news_list = main_area.select('li > a[href], li > div > a[href]') 
    
    # main body에 id 가 존재하였다
    
    article_area = dom.select_one('#articleBodyContents')
    
    if article_area:
        file_name = re.sub(r'\W','',seed.split('/')[-1])           #파일 명에 의미 없는 것 삭제 
        
        with open('TXT/'+file_name+'.txt','w',encoding='utf-8') as f:
            f.write(article_area.text)                                  #Scrapping은 여기서 끝난다.
        
    
    # — 크롤러 역할. 링크를 새로 찾고 추가해주는
    for link in news_list:        
        link = urljoin(seed, link.attrs['href'])
        if link not in seed_list and link not in seen_list:    
            seed_list.append(link)            
        
    
    seed_list.append(seed)

#TXT 폴더에 저장되었다.

InvalidSchema: No connection adapters were found for 'javascript:;'

----------------------

-------

In [ ]:
진짜 가저오고싶은 데이터는 보통 로그인 후 접근할 수 있다 대부분..
이럴땐 DHTML (Dynamic hypertext markup language) 을 사용하면 된다!

SPA lifecycle
initial request ->
html <-
ajax ->
json <-

브런치는 dhtml의 예시


In [98]:
url = 'https://brunch.co.kr/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC'
resp = new_download(url)

False


In [100]:
dom = BeautifulSoup(resp.text, 'lxml')
dom

#resultArticle은 비어있다
#페이지에선 검색할 수 있지만 안보인다 = 못가져온다!

<!DOCTYPE html>
<html lang="ko">
<head>
<title>브런치 글 검색: </title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="브런치 글 검색: " name="title"/>
<meta content="always" name="referrer"/>
<meta name="secureToken" value=""/>
<meta content="max-image-preview:large" name="robots"/>
<meta content="다양한 시선의 글을 검색해보세요!" name="description"/>
<meta content="brunch" property="og:site_name"/>
<meta content="브런치 글 검색: " property="og:title"/>
<meta content="다양한 시선의 글을 검색해보세요!" property="og:description"/>
<meta content="https://t1.daumcdn.net/brunch/static/img/help/pc/img_share_default_800.png" property="og:image"/>
<meta content="https://brunch.co.kr/search?q=&amp;type=article" property="og:url"/>
<meta content="summary" name="twitter:card"/>
<meta content="brunch" name="twitter:app:name:iphone"/>
<meta content="1001388574" name="twitter:app:id:iphone"/>
<meta content="brunchapp://open" name

In [103]:
#request 찾아서 업데이트해보자
#request는 개발자툴 xhr header에 있당
url = 'https://api.brunch.co.kr/v1/search/article?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&page=1&pageSize=20&highlighter=y&escape=y&sortBy=accu'
resp = new_download(url)

404 Client Error: Not Found for url: https://api.brunch.co.kr/robots.txt
False


In [104]:
resp.headers['content-type']

'application/json;charset=UTF-8'

In [105]:
for book in resp.json()['data']['list']:
    print(book.keys())
    break

dict_keys(['id', 'version', 'no', 'userId', 'userName', 'status', 'magazineNo', 'magazineAddress', 'magazineAddressUri', 'magazineArticleId', 'magazineTitle', 'magazineStatus', 'magazineType', 'magazineCoverImage', 'commentWritable', 'commentCount', 'likeCount', 'title', 'subTitle', 'contentSummary', 'contentHash', 'authorComment', 'createTime', 'updateTime', 'publishTime', 'publishTimestamp', 'restrictTime', 'content', 'profileId', 'coverInfo', 'articleImageList', 'socialShareTotalCount', 'coverColor', 'articleKeywords', 'readSeconds', 'fromTime', 'toTime', 'orderBy', 'chapterNo', 'seq', 'tempBrunchbookNo', 'tempBrunchbookTitle', 'tempBrunchbookStatus', 'tempBrunchbookAddress', 'readTime', 'flattenedSeq', 'viewCount', 'kakaoShareCount', 'kakaoTalkShareCount', 'facebookShareCount', 'twitterShareCount', 'contentIdParam', 'homeDataSourceType', 'arrivedLikeitCount', 'profile', 'articleKeywordNameAsCsv', 'articleImageListForHome', 'notPublished', 'contentId', 'pick', 'articleImageForDiscov

In [110]:
for book in resp.json()['data']['list']:
    print(re.sub(r'</?[^>]+?>','',book['title']))

프로그래밍 책을 처음 뗐습니다. - 모던 파이썬 입문 책떼기 후기
Python이 뭐야? 뱀이야? - 파이썬과 친해지기_01
비전공자도 개발자가 될 수 있을까? - 파이썬으로 시작하는 개발자 커리어
향후 금융권 주요 분석도구는 “엑셀+파이썬”이 된다.
14. 통계를 위한 파이썬 라이브러리, 넘파이 (2) - 파이썬으로 데이터 분석하기
12. 파일 읽고 쓰기 - 파이썬으로 데이터 분석하기
[인프런 리프] 파이썬 중급 강의4,5주 차후기
102. 파이썬 1주차- 파이썬 알아보기
[인프런 리프] 파이썬 중급 강의2주 차후기 - 클래스, 메소드 심화
파이썬 몰라도 크롤링 가능합니다 - 마우스 클릭 몇 번으로 경쟁사 후기 분석하기
금융권에 필요한 파이썬 스킬트리 #2.
연오의 파이썬, 두근두근 첫만남 - 연오의파이썬 I 지은이 박연오 I 출판사 인사이트
금융권에 필요한 파이썬 스킬트리 #1.
나만의 100일 프로젝트 완료 기록 - 파이썬과 영어공부
나만의 100일 프로젝트 절반의 기록 - 파이썬과 영어공부
124.(책 후기) 파이썬챌린지. 문제 중심
Python : 신조 - #파이썬 왈
데이콘, Fashion MNIST:의류 클래스 예측 - 파이썬 딥러닝 머신러닝 입문 - 4장 딥러닝 응용
파이썬 기초(1)
간절함과 목표 - 영어공부의 전부 그리고 노션 템플릿 - 영어공부 방법론과 파이썬 초보의 코딩 공부 - 2


In [ ]:
이렇게 가져올 수 있다.
이제 로그인까지 해서 자료 긁고 빠져나오는법.. 불법..? 어쩌라고 재밋는데 나 존나 불법ㅋㅋ엌ㅋㅋ

-----------------------

In [113]:
url = 'http://pythonscraping.com/pages/cookies/login.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'lxml')

In [130]:
form = dom.select_one('form')
input_list = dom.select('form input[name]')
#폼태그를 찾고 인풋태그를 찾아야한다
#폼에 있는 어트리뷰트들을

In [131]:
form.attrs

{'method': 'post', 'action': 'welcome.php'}

In [132]:
urljoin(url, form.attrs['action']), form.attrs['method']

('http://pythonscraping.com/pages/cookies/welcome.php', 'post')

In [133]:
for _ in input_list:
    print(_.attrs)
    
#이걸보면 우리가 보내야 할 파라미터를 알 수 있다.

{'type': 'text', 'name': 'username'}
{'type': 'password', 'name': 'password'}


In [134]:
params = dict()
for _ in input_list:
    params[_.attrs['name']] = None

In [136]:
params

{'username': None, 'password': None}

In [141]:
#만약 계정이 여러개면?
#어카운트 개수만큼 포문을 돌린다

account = [('test1','password'),('test2','password'),]

for _ in account:
    params['username'] = _[0]
    params['password'] = _[1]
    
    resp = download(urljoin(url,form.attrs['action']),
             params = params, method = form.attrs['method'])
    
    
    
    print(resp.text)
    break
    
#쿠키때문에 로그인이 안되는것이다!


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


In [25]:
from requests.sessions import Session
#앞으로는 이 세션도 많이 쓸거다

In [143]:
session = Session()

In [146]:
session.cookies.get_dict()
#쿠키에 아무것도 안들어있다!

{}

In [155]:
resp = session.request(url=urljoin(url,form.attrs['action']),
             data = params, method = form.attrs['method'])

In [156]:
session.cookies.get_dict()

{'loggedin': '1', 'username': 'test1'}

In [161]:
session.request(url=urljoin(url,form.attrs['action']),
             method = form.attrs['method']).text

#처음에 쿠키를 생성하고, 파라미터를 없애서 쿠키를 전달해준다

'\n<h2>Welcome to the Website!</h2>\nWhoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>'

In [162]:
session.cookies.clear()

In [165]:
import json
test_cookie = {'loggedin': '1', 'username': 'test1'}
with open('test_cookie.txt', 'w') as f:
    json.dump(test_cookie, f)

In [166]:
with open('test_cookie.txt', 'w') as f:
    cookie = json.load(f)

UnsupportedOperation: not readable

--------

In [169]:
resp = download('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
dom = BeautifulSoup(resp.text, 'lxml')

In [171]:
form = dom.select_one('form')
input_list = dom.select('input[name]')

In [172]:
form.attrs['action'], form.attrs['method']

('https://nid.naver.com/nidlogin.login', 'POST')

In [173]:
for _ in input_list:
    print(_.attrs['name'],
         _.attrs['value'] if _.has_attr('value') else '입력필요')

localechange 
dynamicKey xBxcltgEIgwKGQ8cTr0o7FPaALzLwzl1Ha_sO6wFeO6uygbIR9rCECJYTai0XBK3EeADZ1Et2Zqxe7misTvnK028Zrh3TM4FhcF16DJtG14
encpw 
enctp 1
svctype 1
smart_LEVEL 1
bvsd 
encnm 
locale ko_KR
url https://www.naver.com
id 
pw 입력필요
nvlong off
nclicks_nsc nid.login_kr
nid_buk none
ncaptchaSplit none
id_error_msg 아이디를 입력해주세요.
pw_error_msg 비밀번호를 입력해주세요.
locale ko_KR
adult_surl_v2 


-------------------------------------

# 7월 9일 금요일

In [19]:
import json

In [36]:
conda install -c conda-forge selenium


Solving environment: done

## Package Plan ##

  environment location: /Users/jonghyunlee/opt/anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    selenium-3.141.0           |py38h5406a74_1002         926 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         926 KB

The following NEW packages will be INSTALLED:

  selenium           conda-forge/osx-64::selenium-3.141.0-py38h5406a74_1002



selenium-3.141.0     | 926 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [37]:
conda install -c conda-forge python-chromedriver-binary


Solving environment: done

## Package Plan ##

  environment location: /Users/jonghyunlee/opt/anaconda3

  added / updated specs:
    - python-chromedriver-binary


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python-chromedriver-binary-92.0.4515.43.0|   py38h50d1736_0         7.2 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         7.2 MB

The following NEW packages will be INSTALLED:

  python-chromedriv~ conda-forge/osx-64::python-chromedriver-binary-92.0.4515.43.0-py38h50d1736_0



python-chromedriver- | 7.2 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [35]:
from selenium.webdriver import Chrome

ModuleNotFoundError: No module named 'selenium'